In [94]:
import pandas as pd
import numpy as np

import os
import datetime

from sklearn import feature_extraction

In [95]:
def load_time_csv(file_path, iden_str):
    
    #load the minute interval time stamps
    #Inputs:
    #       file_path: string to identify where the file locates
    #       iden_str: string to identify the target files from the rest
    #Outputs:
    #       pandas dataframe that joins all the targeted file together.
    
    file_counter = 0
    for filename in os.listdir(file_path):
        if iden_str in filename:
            if file_counter == 0:
                output_df = pd.read_csv(file_path + '/' + filename, names=['Time', 'Open', 'High', 'Low', 'Close'])
                file_counter += 1
            else:
                output_df = pd.concat([output_df, pd.read_csv(file_path + '/' + filename, names=['Time', 'Open', 'High', 'Low', 'Close'])])
                file_counter += 1
    
    return output_df

In [96]:
def load_tweet_csv(file_name):
    tweet_df = pd.read_csv(file_name, encoding='utf-8')
    
    return tweet_df

In [105]:
def remove_special_char(input_string):
    
    output_string = input_string.replace(',', '')
    output_string = output_string.replace('.', '')
    output_string = output_string.replace(':', '')
    output_string = output_string.replace('"', '')
    output_string = output_string.replace('-', '')
    output_string = output_string.replace('!', '')
    output_string = output_string.replace('?', '')
    output_string = output_string.replace('/', ' ')
    output_string = output_string.replace(')', '')
    output_string = output_string.replace('(', '')
    
    return output_string

In [98]:
def remove_url_character(input_string):
    
    string_list = input_string.split()
    
    for idx, string in enumerate(string_list):
        if 'https:' in string:
            string_list[idx] = 'url'
        else:    
            string_list[idx] = remove_special_char(string) 
            
    output_string = " ".join(string_list)
    
    return output_string

In [99]:
def build_vocab(text_corp):
    
    flat_list = []
    for tweet in text_corp:
        tweet_list = tweet.split()
        for word in tweet_list:
            flat_list.append(word)
            
    vocab = sorted(set(flat_list))
    print ('{} unique characters'.format(len(vocab)))
    
    return vocab

In [100]:
def join_tweets(tweet_df):
    
    time_stamp_dict = {}
    for idx, row in tweet_df[::-1].iterrows():
        if row['created_at'] not in time_stamp_dict:
            time_stamp_dict[row['created_at']] = remove_url_character(row['text'].lower())
        else:
            time_stamp_dict[row['created_at']] = time_stamp_dict[row['created_at']] + " " + remove_url_character(row['text'].lower())
    
    return time_stamp_dict

In [54]:
def load_word_embedding():
    embedding_dict = {}
    embedding_file = open('Inputs/glove.twitter.27B.200d.txt', 'r')
    lines = embedding_file.readlines()
    
    line_count = 0
    
    for line in lines:
        line_list = line.split()
        embedding_dict[line_list[0]] = [float(i) for i in line_list[1:]]
        
    return embedding_dict

embedding_dict = load_word_embedding()

In [101]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

In [107]:
def get_word_embedding(word, embedding_dict):
    
    vocab = list(embedding_dict.keys())
    
    try:
        vector = np.array(embedding_dict[word])
    
    except KeyError:
        '''
        final_word = ''
        final_word_dist = 10
        for search_word in vocab:
            dist = levenshtein(search_word, word)
            if dist < final_word_dist:
                final_word = search_word
                final_word_dist = dist
        '''
        vector = np.zeros((1, 200))
    
    return vector

In [131]:
def main():
    label_df = load_time_csv('Inputs', '_12')
    tweet_df = load_tweet_csv('Inputs/trumptweet_updated.csv')
    
    tweet_df = tweet_df[tweet_df['is_retweet'] != True].reset_index()
    
    label_df['Time']= pd.to_datetime(label_df['Time'])
    
    label_time_stamps = list(label_df['Time'])
    closed_number = list(label_df['Close'])
    
    label_time_stamps_dt = []
    for time in label_time_stamps:
        label_time_stamps_dt.append(time.to_pydatetime())
    
    tweet_dict = join_tweets(tweet_df)
    
    '''
    with open('Outputs/trumptweetcorp.txt', 'w') as f:
        for tweet in tweet_corp_list:
            f.write("%s\n" % tweet)
    '''
    
    tweet_time_stamp = list(tweet_dict.keys())
    content = list(tweet_dict.values())
    
    selected_time = []
    selected_text = []
    
    for idx, time_str in enumerate(tweet_time_stamp):
        try:
            time_stamp = datetime.datetime.strptime(time_str, '%m/%d/%y %H:%M')
            if time_stamp in label_time_stamps_dt:
                selected_time.append(time_stamp)
                selected_text.append(content[idx])
                
        except TypeError:
            #print(time_str)
            continue
            
    print(len(selected_time), len(selected_text))
    
    #part where word embedding is done.
    
    embedded_text = []     
    for idx, time in enumerate(selected_time): 
        text_embedding = np.zeros((1, 200))
        text = selected_text[idx]
        print(text)
        
        for word in text.split():
            text_embedding += get_word_embedding(word, embedding_dict)
        
        embedded_text.append(text_embedding.reshape(200,))
    
    feature_array = np.array(embedded_text)
    print(feature_array.shape)
    
    
    #part where one-hot vec is done:
    '''
    vectorizer = feature_extraction.text.CountVectorizer()
    X = vectorizer.fit_transform(selected_text)
    
    
    feature_array = X.toarray()
    
    print(len(vectorizer.get_feature_names()))
    print(feature_array.shape)
    print(np.sum(feature_array, axis=1))
    '''
    
    label_list = []
    trend_list = []
    
    for time in selected_time:
        idx = label_time_stamps_dt.index(time) + 1
        
        label_list.append(closed_number[idx])
        trend_list.append(closed_number[idx-10: idx])
        
    trend_array = np.array(trend_list)
    
    X_array = np.concatenate((trend_array, feature_array), axis=1)
    total_array = np.concatenate((np.array(label_list).reshape(len(label_list), 1), X_array), axis=1)
    
    np.savetxt("Outputs/total_array_original.csv", total_array, delimiter=",", fmt='%f')
    print(total_array.shape)
    
    #print(get_word_embedding('trump)', embedding_dict))
            
    return 0

In [78]:
print(np.zeros((1, 200)))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [132]:
if __name__ == "__main__":
    main()

1916 1916
today we make america great again
such a beautiful and important evening the forgotten man and woman will never be forgotten again we will all come together as never before
love the fact that the small groups of protesters last night have passion for our great country we will all come together and be proud
busy day planned in new york will soon be making some very important decisions on the people who will be running our government
today we express our deepest gratitude to all those who have served in our armed forces #thankavet url
if the election were based on total popular vote i would have campaigned in ny florida and california and won even bigger and more easily
the electoral college is actually genius in that it brings all states including the smaller ones into play campaigning is much different
i am not trying to get top level security clearance for my children this was a typically false news story
the failing @nytimes story is so totally wrong on transition it is goi

republicans must be careful in that the dems own the failed obamacare disaster with its poor coverage and massive premium increases
like the 116% hike in arizona also deductibles are so high that it is practically useless don't let the schumer clowns out of this web
massive increases of obamacare will take place this year and dems are to blame for the mess it will fall of its own weight  be careful
the democrats lead by head clown chuck schumer know how bad obamacare is and what a mess they are in instead of working to fix it they
do the typical political thing and blame the fact is obamacare was a lie from the beginningkeep you doctor keep your plan it is
time for republicans &amp; democrats to get together and come up with a healthcare plan that really works  much less expensive &amp; far better
the dishonest media likes saying that i am in agreement with julian assange  wrong i simply state what he states it is for the people
to make up their own minds as to the truth the media lies

if the ban were announced with a one week notice the bad would rush into our country during that week a lot of bad dudes out there
i have made my decision on who i will nominate for the united states supreme court it will be announced live on tuesday at 800 pm wh
where was all the outrage from democrats and the opposition party the media when our jobs were fleeing our country
nancy pelosi and fake tears chuck schumer held a rally at the steps of the supreme court and mic did not work a messjust like dem party
when will the democrats give us our attorney general and rest of cabinet they should be ashamed of themselves no wonder dc doesn't work
everybody is arguing whether or not it is a ban call it what you want it is about keeping bad people with bad intentions out of country
if uc berkeley does not allow free speech and practices violence on innocent people with a different point of view  no federal funds
congratulations to rex tillerson on being sworn in as our new secretary of state

on international women's day join me in honoring the critical role of women here in america &amp; around the world
linkedin workforce report january and february were the strongest consecutive months for hiring since august and september 2015
weekly address  1100 am at the @whitehouse url url
it is amazing how rude much of the media is to my very hard working representatives be nice you will do much better
obamacare is imploding it is a disaster and 2017 will be the worst year yet by far republicans will come together and save the day
jobs jobs jobs url
does anybody really believe that a reporter who nobody ever heard of went to his mailbox and found my tax returns @nbcnews fake news
can you imagine what the outcry would be if @snoopdogg failing career and all had aimed and fired the gun at president obama jail time
will be going to detroit michigan love today for a big meeting on bringing back car production to state &amp; us already happening
looking forward to a big rally in nashvil

happy birthday to our @flotus melania url url
i received calls from the president of mexico and the prime minister of canada asking to renegotiate nafta rather than terminate i agreed
subject to the fact that if we do not reach a fair deal for all we will then terminate nafta relationships are gooddeal very possible
the democrats want to shut government if we don't bail out puerto rico and give billions to their insurance companies for ocare failure no
i want to help our miners while the democrats are blocking their healthcare i promise to rebuild our military and secure our border democrats want to shut down the government politics what's more important rebuilding our military  or bailing out insurance companies ask the democrats
democrats jeopardizing the safety of our troops to bail out their donors from insurance companies it is time to put #americafirst🇺🇸
democrats used to support border security — now they want illegals to pour through our borders as families prepare for summer v

...

had a very good conversation with the newly elected president of brazil jair bolsonaro who won his race by a substantial margin we agreed that brazil and the united states will work closely together on trade military and everything else excellent call wished him congrats
many gang members and some very bad people are mixed into the caravan heading to our southern border please go back you will not be admitted into the united states unless you go through the legal process this is an invasion of our country and our military is waiting for you
in florida there is a choice between a harvard yale educated man named @rondesantisfl who has been a great congressman and will be a great governor  and a dem who is a thief and who is mayor of poorly run tallahassee said to be one of the most corrupt cities in the country
great job being done by congressman keith rothfus of pennsylvania thank you keith
congressman andy barr of kentucky who just had a great debate with his nancy pelosi run opponent 

great bravery shown by police california highway patrol was on scene within 3 minutes with first officer to enter shot numerous times that sheriff’s sergeant died in the hospital god bless all of the victims and families of the victims thank you to law enforcement
“presidential proclamation addressing mass migration through the southern border of the united states” url
@briankempga ran a great race in georgia – he won it is time to move on
you mean they are just now finding votes in florida and georgia – but the election was on tuesday let’s blame the russians and demand an immediate apology from president putin
just returned from france where much was accomplished in my meetings with world leaders never easy bringing up the fact that the us must be treated fairly which it hasn’t on both military and trade we pay for large portions of other countries military protection
hundreds of billions of dollars for the great privilege of losing hundreds of billions of dollars with these same cou

and gm would not be closing their plants in ohio michigan &amp; maryland get smart congress also the countries that send us cars have taken advantage of the us for decades the president has great power on this issue  because of the gm event it is being studied now
general motors is very counter to what other auto and other companies are doing big steel is opening and renovating plants all over the country auto companies are pouring into the us including bmw which just announced a major new plant the usa is booming
did you ever see an investigation more in search of a crime at the same time mueller and the angry democrats aren’t even looking at the atrocious and perhaps subversive crimes that were committed by crooked hillary clinton and the democrats a total disgrace
when will this illegal joseph mccarthy style witch hunt one that has shattered so many innocent lives ever endor will it just go on forever after wasting more than $40000000 is that possible it has proven only one thingthe

ice border patrol and our military have done a fantastic job of securing our southern border a great wall would be however a far easier &amp; less expensive solution we have already built large new sections &amp; fully renovated others making them like new the democrats
however for strictly political reasons and because they have been pulled so far left do not want border security they want open borders for anyone to come in this brings large scale crime and disease our southern border is now secure and will remain that way
i look forward to my meeting with chuck schumer &amp; nancy pelosi in 2006 democrats voted for a wall and they were right to do so today they no longer want border security they will fight it at all cost and nancy must get votes for speaker but the wall will get built
people do not yet realize how much of the wall including really effective renovation has already been built if the democrats do not give us the votes to secure our country the military will build the r

russia iran syria &amp; many others are not happy about the us leaving despite what the fake news says because now they will have to fight isis and others who they hate without us i am building by far the most powerful military in the world isis hits us they are doomed
the democrats who know steel slats wall are necessary for border security are putting politics over country what they are just beginning to realize is that i will not sign any of their legislation including infrastructure unless it has perfect border security usa wins
with so much talk about the wall people are losing sight of the great job being done on our southern border by border patrol ice and our great military remember the caravans well they didn’t get through and none are forming or on their way border is tight fake news silent
when i begrudgingly signed the omnibus bill i was promised the wall and border security by leadership would be done by end of year now it didn’t happen we foolishly fight for border securi